# Make composite images from lots of Trove newspaper thumbnails
This notebook starts with a search in Biblioteca Virtual Miguel de Cervantes. It uses the SPARQL API from data.cervantesvirtual.com/sparql to retrieve the search results. Based on an a list of authors, it creates a thumbnail for each work getting its cover obtaining as a result a folder full of thumbnails.

The next stage is creating a mosaic taking the thumbnails.

The SPARQL query is configured to retrieve the covers from the authors [Miguel de Cervantes](http://data.cervantesvirtual.com/person/40), [Lope de Vega](http://data.cervantesvirtual.com/person/72) y [Calderón de la Barca](http://data.cervantesvirtual.com/person/79). The number of results was configured to 2500 in the SPARQL to retrieve enough covers.

It is important to take into account the size of the images and the number of cols and rows.

## Setting up things

In [65]:
import os
import requests
import pandas as pd
from io import BytesIO
from PIL import Image, ImageDraw, ImageFont, ImageOps
from tqdm import tqdm_notebook

## Set your parameters

In [66]:
size = 120 # Size of the thumbnails
cols = 55 # The width of the final image will be cols x size
rows = 35 # The height of the final image will be cols x size

## Define some functions

In [69]:
def create_composite(cols, rows, size):
    im = Image.new('RGB', (cols*size, rows*size))
    thumbs = [t for t in os.listdir('covers') if t[-4:] == '.jpg']
    x = 0
    y = 0
    
    for index, thumb_file in tqdm_notebook(enumerate(thumbs, 1)):
        
        thumb = Image.open('covers/{}'.format(thumb_file))
        
        th = Image.open('covers/'+thumb_file)
        th.thumbnail((size, size))
        
        new_w, new_h = th.size
        
        # Squarify
        delta_w = size - new_w
        delta_h = size - new_h
        padding = (delta_w//2, delta_h//2, delta_w-(delta_w//2), delta_h-(delta_h//2))
        th = ImageOps.expand(th, padding, fill='white')
        
        th.save("thumbs/thumbnail_"+str(size)+"_"+thumb_file)
        
        try:
            im.paste(th, (x, y, x+size, y+size))
        except ValueError:
            pass
        else:
            if (index % cols) == 0:
                x = 0
                y += size
            else:
                x += size
    im.save('composite-{}-{}.jpg'.format(cols, rows), quality=90)

## Getting the thumbnails

In [14]:
url = 'http://data.cervantesvirtual.com/bvmc-lod/repositories/data'
query = """
PREFIX rdam: <http://rdaregistry.info/Elements/m/> 
PREFIX rdaw: <http://rdaregistry.info/Elements/w/> 
PREFIX dc: <http://purl.org/dc/elements/1.1/> 
PREFIX foaf: <http://xmlns.com/foaf/0.1/> 
PREFIX dcterms: <http://purl.org/dc/terms/> 
SELECT ?m ?cover ?label 
WHERE { ?m rdfs:label ?label . 
?m foaf:depiction ?cover . 
  ?m rdam:workManifested ?w .
  {
   { ?w rdaw:author <http://data.cervantesvirtual.com/person/40>} 
     UNION
   { ?w rdaw:author <http://data.cervantesvirtual.com/person/72>}
     UNION
   { ?w rdaw:author <http://data.cervantesvirtual.com/person/79>}
  }
} OFFSET 0 LIMIT 100 
"""
r = requests.get(url, params = {'format': 'text/plain', 'query': query})

# save the result
f = open("results.csv", "w")
f.write(r.text)
f.close()

Reading the results to get the covers

In [15]:
# Load the CSV file from GitHub.
# This puts the data in a Pandas DataFrame
df = pd.read_csv('results.csv')

In [16]:
# Let's have a look inside...
# Note that both the columns and rows are truncated in this preview
df

,m,cover,label
0,http://data.cervantesvirtual.com/manifestation...,http://media.cervantesvirtual.com/s3/BVMC_OBRA...,El Ingenioso hidalgo Don Quixote de la Mancha
1,http://data.cervantesvirtual.com/manifestation...,http://media.cervantesvirtual.com/s3/BVMC_OBRA...,El Ingenioso hidalgo Don Quixote de la Mancha
2,http://data.cervantesvirtual.com/manifestation...,http://media.cervantesvirtual.com/s3/BVMC_OBRA...,El Ingenioso hidalgo Don Quixote de la Mancha
3,http://data.cervantesvirtual.com/manifestation...,http://media.cervantesvirtual.com/s3/BVMC_OBRA...,El capitán Becerra vino a Sevilla a enseñar lo...
4,http://data.cervantesvirtual.com/manifestation...,http://media.cervantesvirtual.com/s3/BVMC_OBRA...,El ingenioso hidalgo don Quixote de la Mancha ...
5,http://data.cervantesvirtual.com/manifestation...,http://media.cervantesvirtual.com/s3/BVMC_OBRA...,"Vida, y hechos del ingenioso cavallero D. Quix..."
6,http://data.cervantesvirtual.com/manifestation...,http://media.cervantesvirtual.com/s3/BVMC_OBRA...,"Vida, y hechos del ingenioso cavallero D. Quix..."
7,http://data.cervantesvirtual.com/manifestation...,http://media.cervantesvirtual.com/s3/BVMC_OBRA...,"Sonetos en ""Don Quijote de la Mancha"" ( 1605) ..."
8,http://data.cervantesvirtual.com/manifestation...,http://media.cervantesvirtual.com/s3/BVMC_OBRA...,Sonetos dedicados a sus amigos y otros / Migue...
9,http://data.cervantesvirtual.com/manifestation...,http://media.cervantesvirtual.com/s3/BVMC_OBRA...,"Sonetos de comedias, novelas, entremeses y otr..."


## Retrieving the covers

In [17]:
for index, row in df.iterrows():
    print(index, row['cover'])
    response = requests.get(row['cover'])
    img = Image.open(BytesIO(response.content))
    img.save('covers/m-{}.jpg'.format(row['m'].replace('http://data.cervantesvirtual.com/manifestation/','')), quality=90)

0 http://media.cervantesvirtual.com/s3/BVMC_OBRAS//2e5/cf6/23e/82b/479/498/282/797/e65/4a6/e8/portada/Cover.jpg
1 http://media.cervantesvirtual.com/s3/BVMC_OBRAS//7ba/29b/37c/8b8/48c/0ad/890/44a/367/1d2/f9/portada/Cover.jpg
2 http://media.cervantesvirtual.com/s3/BVMC_OBRAS//ee7/287/567/db7/449/b8e/408/086/84f/49c/19/portada/Cover.jpg
3 http://media.cervantesvirtual.com/s3/BVMC_OBRAS//ffd/4f1/608/2b1/11d/fac/c70/021/85c/e60/64/portada/Cover.jpg
4 http://media.cervantesvirtual.com/s3/BVMC_OBRAS//5b8/851/fca/1d1/431/dba/7f2/6b8/29e/01f/2a/portada/Cover.jpg
5 http://media.cervantesvirtual.com/s3/BVMC_OBRAS//402/330/b2c/2fc/459/889/0a6/c9d/1b7/cc0/c6/portada/Cover.jpg
6 http://media.cervantesvirtual.com/s3/BVMC_OBRAS//b1d/4b3/50a/07a/46b/296/074/80a/4bb/5a6/17/portada/Cover.jpg
7 http://media.cervantesvirtual.com/s3/BVMC_OBRAS//ffd/647/d68/2b1/11d/fac/c70/021/85c/e60/64/portada/Cover.jpg
8 http://media.cervantesvirtual.com/s3/BVMC_OBRAS//ffd/64f/7e8/2b1/11d/fac/c70/021/85c/e60/64/portada/Co

73 http://media.cervantesvirtual.com/s3/BVMC_OBRAS//ff3/21a/9e8/2b1/11d/fac/c70/021/85c/e60/64/portada/Cover.jpg
74 http://media.cervantesvirtual.com/s3/BVMC_OBRAS//ff3/222/508/2b1/11d/fac/c70/021/85c/e60/64/portada/Cover.jpg
75 http://media.cervantesvirtual.com/s3/BVMC_OBRAS//ff3/27a/528/2b1/11d/fac/c70/021/85c/e60/64/portada/Cover.jpg
76 http://media.cervantesvirtual.com/s3/BVMC_OBRAS//ff3/282/ea8/2b1/11d/fac/c70/021/85c/e60/64/portada/Cover.jpg
77 http://media.cervantesvirtual.com/s3/BVMC_OBRAS//ff3/293/528/2b1/11d/fac/c70/021/85c/e60/64/portada/Cover.jpg
78 http://media.cervantesvirtual.com/s3/BVMC_OBRAS//ff3/7ca/348/2b1/11d/fac/c70/021/85c/e60/64/portada/Cover.jpg
79 http://media.cervantesvirtual.com/s3/BVMC_OBRAS//ff3/29b/2c8/2b1/11d/fac/c70/021/85c/e60/64/portada/Cover.jpg
80 http://media.cervantesvirtual.com/s3/BVMC_OBRAS//ff3/2b9/ea8/2b1/11d/fac/c70/021/85c/e60/64/portada/Cover.jpg
81 http://media.cervantesvirtual.com/s3/BVMC_OBRAS//ff3/6fb/728/2b1/11d/fac/c70/021/85c/e60/64/p

146 http://media.cervantesvirtual.com/s3/BVMC_OBRAS//ff9/8b1/008/2b1/11d/fac/c70/021/85c/e60/64/portada/Cover.jpg
147 http://media.cervantesvirtual.com/s3/BVMC_OBRAS//ff9/8b8/b28/2b1/11d/fac/c70/021/85c/e60/64/portada/Cover.jpg
148 http://media.cervantesvirtual.com/s3/BVMC_OBRAS//ff9/8c3/7a8/2b1/11d/fac/c70/021/85c/e60/64/portada/Cover.jpg
149 http://media.cervantesvirtual.com/s3/BVMC_OBRAS//ff9/8cb/368/2b1/11d/fac/c70/021/85c/e60/64/portada/Cover.jpg
150 http://media.cervantesvirtual.com/s3/BVMC_OBRAS//ff9/8d2/e88/2b1/11d/fac/c70/021/85c/e60/64/portada/Cover.jpg
151 http://media.cervantesvirtual.com/s3/BVMC_OBRAS//ff9/8db/1c8/2b1/11d/fac/c70/021/85c/e60/64/portada/Cover.jpg
152 http://media.cervantesvirtual.com/s3/BVMC_OBRAS//ff9/8e2/d88/2b1/11d/fac/c70/021/85c/e60/64/portada/Cover.jpg
153 http://media.cervantesvirtual.com/s3/BVMC_OBRAS//ff9/8ea/8a8/2b1/11d/fac/c70/021/85c/e60/64/portada/Cover.jpg
154 http://media.cervantesvirtual.com/s3/BVMC_OBRAS//ff9/adf/8e8/2b1/11d/fac/c70/021/85c

218 http://media.cervantesvirtual.com/s3/BVMC_OBRAS//35d/dda/72e/c02/4de/8bc/c17/c62/b91/36e/bd/portada/Cover.jpg
219 http://media.cervantesvirtual.com/s3/BVMC_OBRAS//003/584/588/2b2/11d/fac/c70/021/85c/e60/64/portada/Cover.jpg
220 http://media.cervantesvirtual.com/s3/BVMC_OBRAS//003/bb8/148/2b2/11d/fac/c70/021/85c/e60/64/portada/Cover.jpg
221 http://media.cervantesvirtual.com/s3/BVMC_OBRAS//003/e8f/308/2b2/11d/fac/c70/021/85c/e60/64/portada/Cover.jpg
222 http://media.cervantesvirtual.com/s3/BVMC_OBRAS//003/e97/8c8/2b2/11d/fac/c70/021/85c/e60/64/portada/Cover.jpg
223 http://media.cervantesvirtual.com/s3/BVMC_OBRAS//ffc/689/ea8/2b1/11d/fac/c70/021/85c/e60/64/portada/Cover.jpg
224 http://media.cervantesvirtual.com/s3/BVMC_OBRAS//004/20a/de8/2b2/11d/fac/c70/021/85c/e60/64/portada/Cover.jpg
225 http://media.cervantesvirtual.com/s3/BVMC_OBRAS//004/21a/2e8/2b2/11d/fac/c70/021/85c/e60/64/portada/Cover.jpg
226 http://media.cervantesvirtual.com/s3/BVMC_OBRAS//004/221/d68/2b2/11d/fac/c70/021/85c

290 http://media.cervantesvirtual.com/s3/BVMC_OBRAS//099/fa6/b7d/ec0/4dd/da7/792/15d/496/b07/59/portada/Cover.jpg
291 http://media.cervantesvirtual.com/s3/BVMC_OBRAS//4ac/8e9/528/872/44b/c87/869/0a6/65f/7bb/0d/portada/Cover.jpg
292 http://media.cervantesvirtual.com/s3/BVMC_OBRAS//4b8/5f0/1f5/74a/404/e9b/bd3/f97/fa1/6b4/96/portada/Cover.jpg
293 http://media.cervantesvirtual.com/s3/BVMC_OBRAS//725/19c/1cc/851/41e/79f/07e/78d/aeb/1d2/57/portada/Cover.jpg
294 http://media.cervantesvirtual.com/s3/BVMC_OBRAS//4e8/b3f/caa/b2c/4e0/885/dc0/cd2/942/1eb/d4/portada/Cover.jpg
295 http://media.cervantesvirtual.com/s3/BVMC_OBRAS//632/e6d/8db/5c3/41b/4af/22a/30f/d70/c45/8b/portada/Cover.jpg
296 http://media.cervantesvirtual.com/s3/BVMC_OBRAS//566/227/3db/6b2/430/0b8/acb/b96/78b/ba9/24/portada/Cover.jpg
297 http://media.cervantesvirtual.com/s3/BVMC_OBRAS//de2/eff/645/839/42a/4b1/5fb/e08/6f8/891/cb/portada/Cover.jpg
298 http://media.cervantesvirtual.com/s3/BVMC_OBRAS//173/8b1/6e8/81d/4e8/0aa/c52/bac/b31

...

2090 http://media.cervantesvirtual.com/s3/BVMC_OBRAS//d34/680/3db/feb/433/5a2/662/7a6/37b/7d9/db/portada/Cover.jpg
2091 http://media.cervantesvirtual.com/s3/BVMC_OBRAS//b51/1fe/c6c/812/400/fa8/4df/009/e58/09a/8b/portada/Cover.jpg
2092 http://media.cervantesvirtual.com/s3/BVMC_OBRAS//317/3db/2df/5b6/48e/eab/88d/c03/e96/487/bf/portada/Cover.jpg
2093 http://media.cervantesvirtual.com/s3/BVMC_OBRAS//be2/5f7/61a/bd5/45c/186/f0e/ca5/558/5d0/fd/portada/Cover.jpg
2094 http://media.cervantesvirtual.com/s3/BVMC_OBRAS//9a3/ea4/f23/a27/4a0/69b/2a7/628/32e/ad4/84/portada/Cover.jpg
2095 http://media.cervantesvirtual.com/s3/BVMC_OBRAS//573/855/481/edf/456/1aa/32c/431/5d7/eb1/ab/portada/Cover.jpg
2096 http://media.cervantesvirtual.com/s3/BVMC_OBRAS//09c/6d0/002/2cc/4d0/69c/275/f1e/b0c/57b/11/portada/Cover.jpg
2097 http://media.cervantesvirtual.com/s3/BVMC_OBRAS//1ec/074/f10/c30/47b/986/16d/1ab/063/4cd/ee/portada/Cover.jpg
2098 http://media.cervantesvirtual.com/s3/BVMC_OBRAS//8d5/630/06f/945/4b1/1a0/6d

2162 http://media.cervantesvirtual.com/s3/BVMC_OBRAS//bf4/80a/6e9/3f3/4b1/4a0/d21/6ae/683/652/77/portada/Cover.jpg
2163 http://media.cervantesvirtual.com/s3/BVMC_OBRAS//0ed/582/3f6/e42/42d/587/5f4/179/d30/f6b/a8/portada/Cover.jpg
2164 http://media.cervantesvirtual.com/s3/BVMC_OBRAS//5f8/3c2/712/74b/451/a9e/0d1/883/30b/17a/a6/portada/Cover.jpg
2165 http://media.cervantesvirtual.com/s3/BVMC_OBRAS//209/710/66e/936/4de/895/e10/57b/e4a/509/fc/portada/Cover.jpg
2166 http://media.cervantesvirtual.com/s3/BVMC_OBRAS//db0/139/990/fc8/430/eac/422/527/26b/ceb/bd/portada/Cover.jpg
2167 http://media.cervantesvirtual.com/s3/BVMC_OBRAS//f71/ff1/062/53a/424/ab7/a8e/352/888/db4/66/portada/Cover.jpg
2168 http://media.cervantesvirtual.com/s3/BVMC_OBRAS//3b8/f2a/278/5ef/4c5/687/b85/319/cd1/665/c3/portada/Cover.jpg
2169 http://media.cervantesvirtual.com/s3/BVMC_OBRAS//0e4/7a3/aed/50b/4b8/d8a/83d/fc4/9b7/e13/e7/portada/Cover.jpg
2170 http://media.cervantesvirtual.com/s3/BVMC_OBRAS//4ff/93c/01a/5a6/42e/686/73

2234 http://media.cervantesvirtual.com/s3/BVMC_OBRAS//f3e/661/152/f49/4c5/ea9/dbf/ac9/11a/f87/9a/portada/Cover.jpg
2235 http://media.cervantesvirtual.com/s3/BVMC_OBRAS//c77/584/8c7/75e/4e1/aa1/21a/4e2/5c7/cc9/51/portada/Cover.jpg
2236 http://media.cervantesvirtual.com/s3/BVMC_OBRAS//f79/e04/4ac/795/40d/396/9d2/c60/6a7/d69/6b/portada/Cover.jpg
2237 http://media.cervantesvirtual.com/s3/BVMC_OBRAS//96f/6a4/3f7/bad/47b/3a5/0ce/c2a/9cb/3b6/3c/portada/Cover.jpg
2238 http://media.cervantesvirtual.com/s3/BVMC_OBRAS//b03/387/81e/154/4e8/2a2/beb/9c4/230/98d/db/portada/Cover.jpg
2239 http://media.cervantesvirtual.com/s3/BVMC_OBRAS//d57/093/b13/3a1/444/6a7/54f/7cb/4aa/9a0/dc/portada/Cover.jpg
2240 http://media.cervantesvirtual.com/s3/BVMC_OBRAS//c26/4f5/46f/18d/4dc/c83/73b/d6c/071/bf4/71/portada/Cover.jpg
2241 http://media.cervantesvirtual.com/s3/BVMC_OBRAS//0eb/7fc/119/134/4bc/fb0/629/a05/43e/498/45/portada/Cover.jpg
2242 http://media.cervantesvirtual.com/s3/BVMC_OBRAS//d01/7e3/433/09c/474/99b/b7

2306 http://media.cervantesvirtual.com/s3/BVMC_OBRAS//bc1/def/9f6/0b9/4be/eb4/17a/ad9/115/6d3/fe/portada/Cover.jpg
2307 http://media.cervantesvirtual.com/s3/BVMC_OBRAS//a53/387/1af/c9e/430/9a3/6a0/28f/6a6/cbd/90/portada/Cover.jpg
2308 http://media.cervantesvirtual.com/s3/BVMC_OBRAS//0b0/487/fe0/27f/415/282/b08/a74/8c3/c1b/1f/portada/Cover.jpg
2309 http://media.cervantesvirtual.com/s3/BVMC_OBRAS//874/8ff/2f0/176/4f6/784/cf8/92f/042/ce5/32/portada/Cover.jpg
2310 http://media.cervantesvirtual.com/s3/BVMC_OBRAS//aca/c38/003/98b/471/e87/c4c/e12/7e1/9cb/bb/portada/Cover.jpg
2311 http://media.cervantesvirtual.com/s3/BVMC_OBRAS//5ff/171/305/c1e/433/cbd/e0e/92b/041/61b/fd/portada/Cover.jpg
2312 http://media.cervantesvirtual.com/s3/BVMC_OBRAS//8f5/6bc/47f/4fa/47a/4b1/ae6/d0d/379/5e5/ad/portada/Cover.jpg
2313 http://media.cervantesvirtual.com/s3/BVMC_OBRAS//5bd/cba/984/863/489/0af/3bf/f90/dc2/25c/58/portada/Cover.jpg
2314 http://media.cervantesvirtual.com/s3/BVMC_OBRAS//54c/1bc/87d/8c4/4ce/d9c/e1

2378 http://media.cervantesvirtual.com/s3/BVMC_OBRAS//6d0/489/0a1/7b4/436/183/e52/957/f3d/658/48/portada/Cover.jpg
2379 http://media.cervantesvirtual.com/s3/BVMC_OBRAS//9c4/289/b7d/3ed/450/2be/448/9da/a48/80f/cc/portada/Cover.jpg
2380 http://media.cervantesvirtual.com/s3/BVMC_OBRAS//7f4/ebe/fa7/af3/48a/193/972/d99/446/4e6/0d/portada/Cover.jpg
2381 http://media.cervantesvirtual.com/s3/BVMC_OBRAS//9c8/7b1/8b7/60c/421/2b2/11f/62e/92c/68e/36/portada/Cover.jpg
2382 http://media.cervantesvirtual.com/s3/BVMC_OBRAS//a24/e5f/be0/10d/4e0/d8c/1c4/af0/e6f/851/ea/portada/Cover.jpg
2383 http://media.cervantesvirtual.com/s3/BVMC_OBRAS//a16/9c3/b52/79a/4d3/1ab/a46/a0a/3a1/318/a8/portada/Cover.jpg
2384 http://media.cervantesvirtual.com/s3/BVMC_OBRAS//af2/fca/921/ccb/436/9a1/bbf/d6d/791/914/77/portada/Cover.jpg
2385 http://media.cervantesvirtual.com/s3/BVMC_OBRAS//d7c/c74/b76/b4b/418/eaf/e62/607/fa8/3a4/61/portada/Cover.jpg
2386 http://media.cervantesvirtual.com/s3/BVMC_OBRAS//4fe/aca/8e9/873/11e/1b1/fb

## Generating the mosaic

In [70]:
create_composite(cols, rows, size)

85
85
85
85
85
87
87
85
85
85
85
85
91
84
85
85
88
81
85
85
92
85
88
85
68
85
85
85
85
85
85
88
85
85
85
85
85
88
87
85
85
85
85
87
85
88
86
81
85
85
85
85
85
83
85
88
85
85
85
84
85
85
84
82
85
85
85
85
85
85
85
85
85
85
84
88
86
85
85
85
91
85
91
85
91
85
85
85
85
85
84
84
91
89
85
85
88
85
85
85
88
85
85
85
86
85
68
85
88
92
85
85
83
85
85
85
90
85
85
85
85
85
85
82
90
87
85
85
92
90
85
82
85
89
86
85
84
85
85
85
85
85
83
85
85
85
85
85
85
85
85
82
85
80
85
85
87
85
85
87
88
85
86
85
85
74
85
85
85
83
85
85
85
85
87
85
82
85
85
85
85
85
85
85
85
85
85
87
85
85
85
85
85
85
83
85
85
85
85
85
86
85
91
85
85
81
85
89
85
85
85
85
85
87
85
88
81
85
91
86
85
85
83
85
85
85
84
85
87
85
85
85
86
85
88
84
82
93
87
89
85
85
88
83
85
85
85
85
85
87
79
85
85
85
85
85
86
85
85
81
87
87
84
85
85
85
87
85
85
85
88
85
85
88
85
87
85
85
81
85
85
85
85
85
83
85
85
85
83
85
90
86
85
87
82
87
85
88
82
85
85
85
85
85
85
85
85
85
82
85
87
90
81
87
85
73
89
85
84
85
88
89
86
81
85
85
85
83
85
85
85
91
85
8